# Introduction

Fetching the data from AEX for all the stocks scraped from the list acquired on wikipedia

## Preamble

In [1]:
import os
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf
import requests
from bs4 import BeautifulSoup

In [2]:
print(yf.__version__)  # Prints the version of pandas

0.2.37


In [3]:
cwd = os.getcwd()

In [4]:
#print(dir(yf))

In [5]:
#print(dir(yf.Ticker))

In [6]:
#print(dir(yf.Tickers))

## Web Scraper

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the Wikipedia page for the AEX index
url = "https://nl.wikipedia.org/wiki/AEX"

# Send a GET request to the Wikipedia page
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all tables on the page
    tables = soup.find_all("table", class_="wikitable sortable")
    
    # Check if there are at least two tables
    if len(tables) >= 2:
        # Extract the second table (index 1)
        table = tables[1]
        
        # Extract the rows from the table
        rows = table.find_all("tr")
        
        # Extract the column headers
        headers = [header.text.strip() for header in rows[0].find_all("th")]
        
        # Extract the data from the table rows
        data = []
        for row in rows[1:]:
            data.append([cell.text.strip() for cell in row.find_all("td")])
        
        # Convert the data to a DataFrame
        df = pd.DataFrame(data, columns=headers)
        
        # Print the DataFrame
        #print("DataFrame for the second table on AEX index:")
        #print(df)
    else:
        print("Second table not found on the page.")
else:
    print("Failed to retrieve data from Wikipedia.")


In [8]:
print(df["Ticker symbol"].tolist())

['ABN', 'ADYEN', 'AGN', 'AD', 'AKZA', 'MT', 'ASM', 'ASML', 'ASRNL', 'BESI', 'DSM', 'EXOR', 'HEIA', 'IMCD', 'INGA', 'KPN', 'NN', 'PHIA', 'PRX', 'RAND', 'REN', 'RDSA', 'UNA', 'UMG', 'WKL']


## Stocks Fetcher

In [9]:
# List of AEX stocks
aex_stocks = df["Ticker symbol"].tolist()
start_date ='2022-01-01'
end_date = '2022-12-31'

# Directory to save the dataframes
output_directory = "aex_data/"

# Create the output directory if it doesn't exist
try:
    os.makedirs(output_directory)
    print(f"Output directory '{output_directory}' created successfully.")
except FileExistsError:
    print(f"Output directory '{output_directory}' already exists.")

os_path = os.path.join(cwd, output_directory)
# Download historical data for each stock
failed_stocks = []
for stock in aex_stocks:
    try:
        data = yf.download(stock + ".AS", start='2022-01-01', end='2022-12-31')  # Append ".AS" for Amsterdam exchange
        if not data.empty:
            print(f"Successfully downloaded data for {stock}")
            # Save the dataframe to CSV
            file_path = os.path.join(os_path, f"{stock}_data_{start_date}_{end_date}.csv")
            data.to_csv(file_path,mode='w', index=False)
            print(f"Data saved for {stock}. To {file_path} with overwriting permissions'")
            #print(data.head())  # Example: Print the first few rows of data
        else:
            print(f"Downloaded data for {stock} is empty.")
            failed_stocks.append(stock)
    except Exception as e:
        print(f"Failed to download data for {stock}: {e}")
        failed_stocks.append(stock)

# Print failed stocks
print("Failed to download data for the following stocks:", failed_stocks)


Output directory 'aex_data/' already exists.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Successfully downloaded data for ABN
Data saved for ABN. To C:\Users\hugos\Desktop\PersonalProject\predictiveForecasting\aex_data/ABN_data_2022-01-01_2022-12-31.csv with overwriting permissions'
Successfully downloaded data for ADYEN
Data saved for ADYEN. To C:\Users\hugos\Desktop\PersonalProject\predictiveForecasting\aex_data/ADYEN_data_2022-01-01_2022-12-31.csv with overwriting permissions'


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Successfully downloaded data for AGN
Data saved for AGN. To C:\Users\hugos\Desktop\PersonalProject\predictiveForecasting\aex_data/AGN_data_2022-01-01_2022-12-31.csv with overwriting permissions'
Successfully downloaded data for AD
Data saved for AD. To C:\Users\hugos\Desktop\PersonalProject\predictiveForecasting\aex_data/AD_data_2022-01-01_2022-12-31.csv with overwriting permissions'


...

Successfully downloaded data for RAND
Data saved for RAND. To C:\Users\hugos\Desktop\PersonalProject\predictiveForecasting\aex_data/RAND_data_2022-01-01_2022-12-31.csv with overwriting permissions'
Successfully downloaded data for REN
Data saved for REN. To C:\Users\hugos\Desktop\PersonalProject\predictiveForecasting\aex_data/REN_data_2022-01-01_2022-12-31.csv with overwriting permissions'


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['RDSA.AS']: Exception('%ticker%: No timezone found, symbol may be delisted')
[*********************100%%**********************]  1 of 1 completed


Downloaded data for RDSA is empty.
Successfully downloaded data for UNA
Data saved for UNA. To C:\Users\hugos\Desktop\PersonalProject\predictiveForecasting\aex_data/UNA_data_2022-01-01_2022-12-31.csv with overwriting permissions'


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Successfully downloaded data for UMG
Data saved for UMG. To C:\Users\hugos\Desktop\PersonalProject\predictiveForecasting\aex_data/UMG_data_2022-01-01_2022-12-31.csv with overwriting permissions'
Successfully downloaded data for WKL
Data saved for WKL. To C:\Users\hugos\Desktop\PersonalProject\predictiveForecasting\aex_data/WKL_data_2022-01-01_2022-12-31.csv with overwriting permissions'
Failed to download data for the following stocks: ['DSM', 'EXOR', 'RDSA']
